In [2]:
from abc import ABC
from typing import Tuple

import chex
import jumanji
from jumanji import specs
from jumanji.env import State
from jumanji.types import TimeStep

In [5]:
from bin.game_interfaces_binding import *

In [1]:
class ProconJumanji(jumanji.Environment):
    def __init__(self, gameOptions: GameOptions, map, craftsmen):
        self.maxTurns = gameOptions.maxTurns
        self.mapWidth = gameOptions.mapWidth
        self.mapHeight = gameOptions.mapHeight

        self.wallCoeff = gameOptions.wallCoeff
        self.castleCoeff = gameOptions.castleCoeff
        self.territoryCoeff = gameOptions.territoryCoeff

        self.map = map
        self.craftsmen = craftsmen
        self.game = Game(gameOptions, map, craftsmen)

    def reset(self, key: chex.PRNGKey) -> Tuple[State, TimeStep]:
        self.game.reset()
        

    def step(self, state: State, action: chex.Array) -> Tuple[State, TimeStep]:
        self.game.step(action)

    def observation_spec(self) -> specs.Spec:
        return 

    def action_spec(self) -> specs.Spec:
        pass


    def render(self, state: State) -> Any:
        """Render frames of the environment for a given state.

        Args:
            state: State object containing the current dynamics of the environment.
        """
        raise NotImplementedError("Render method not implemented for this environment.")
    

    # def reward_spec(self) -> specs.Array:
    #     """Describes the reward returned by the environment. By default, this is assumed to be a
    #     single float.

    #     Returns:
    #         reward_spec: a `specs.Array` spec.
    #     """
    #     return specs.Array(shape=(), dtype=float, name="reward")

    # def discount_spec(self) -> specs.BoundedArray:
    #     """Describes the discount returned by the environment. By default, this is assumed to be a
    #     single float between 0 and 1.

    #     Returns:
    #         discount_spec: a `specs.BoundedArray` spec.
    #     """
    #     return specs.BoundedArray(
    #         shape=(), dtype=float, minimum=0.0, maximum=1.0, name="discount"
    #     )





NameError: name 'jumanji' is not defined